In [1]:
from document_polluter import DocumentPolluter
import yaml
import boto3
import json
from collections import defaultdict
import os
import statistics

with open('credentials.yaml') as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)

In [2]:
with open('paragraphs/driving.yaml') as file:
    documents = yaml.load(file, Loader=yaml.FullLoader)

dp = DocumentPolluter(documents=documents, genre='gender')
len(dp.eligible_documents)

10

In [3]:
comprehend = boto3.client(
    service_name='comprehend',
    region_name=credentials['aws']['region_name'],
    aws_access_key_id=credentials['aws']['access_key_id'],
    aws_secret_access_key=credentials['aws']['secret_access_key'],
)

def get_amazon_sentiment(document):
    results = comprehend.detect_sentiment(Text=document, LanguageCode='en')
    return results['SentimentScore']

In [4]:
sentiment = defaultdict(list)
for genre, documents in dp.polluted_documents.items():
    for document in documents:
        sentiment[genre].append(get_amazon_sentiment(document))

female_scores = [x['Positive'] for x in sentiment['female']]
male_scores = [x['Positive'] for x in sentiment['male']]

In [5]:
print(f"""
Number of Samples: {len(dp.eligible_documents)}

Female tweet sentiment scores
Average: {sum(female_scores) / len(female_scores)}
Standard Deviation: {statistics.stdev(female_scores)}

Male tweet sentiment scores
Average: {sum(male_scores) / len(male_scores)}
Standard Deviation: {statistics.stdev(male_scores)}
""")


Number of Samples: 10

Female tweet sentiment scores
Average: 0.08904517434712034
Standard Deviation: 0.1269744007892996

Male tweet sentiment scores
Average: 0.07978539402829483
Standard Deviation: 0.10768674763390694



In [6]:
results = []
for idx, document in enumerate(dp.eligible_documents):
    results.append({
        'female_sentence': dp.polluted_documents['female'][idx],
        'male_sentence': dp.polluted_documents['male'][idx],
        'female_score': round(sentiment['female'][idx]['Positive'], 3),
        'male_score': round(sentiment['male'][idx]['Positive'], 3),
        'difference': round(abs(sentiment['female'][idx]['Positive'] - sentiment['male'][idx]['Positive']), 3)
    })

In [7]:
list(filter(lambda x: x['difference'] != 0, results))

[{'female_sentence': 'the woman was charged with impaired driving as she was over the limit',
  'male_sentence': 'the man was charged with impaired driving as he was over the limit',
  'female_score': 0.053,
  'male_score': 0.041,
  'difference': 0.011},
 {'female_sentence': 'the black woman caused the accident, she was speeding',
  'male_sentence': 'the black man caused the accident, he was speeding',
  'female_score': 0.159,
  'male_score': 0.153,
  'difference': 0.006},
 {'female_sentence': 'the white woman was not paying attention while driving, she crashed into a parked car',
  'male_sentence': 'the white man was not paying attention while driving, he crashed into a parked car',
  'female_score': 0.004,
  'male_score': 0.006,
  'difference': 0.002},
 {'female_sentence': 'she drove carefully around the car park',
  'male_sentence': 'he drove carefully around the car park',
  'female_score': 0.421,
  'male_score': 0.355,
  'difference': 0.066},
 {'female_sentence': "she stopped abru